### Установка и импорт необходимых пакетов

In [1]:
!pip install pymorphy2[fast] &> /dev/null
!pip install pymystem3  &> /dev/null

import pandas as pd
import re
import nltk
import pymorphy2
from pymystem3 import Mystem
from nltk.corpus import stopwords
from nltk import FreqDist

nltk.download('stopwords')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

### Обработка текста

In [2]:
df = pd.read_csv('financial_review.csv')

In [3]:
df.shape

(23810, 10)

In [4]:
df['bank_name'].value_counts()[:10]

втб                          2603
сбербанк                     2481
тинькофф банк                1476
почта банк                   1264
альфа-банк                   1260
открытие                      868
совкомбанк                    844
отп банк                      698
московский кредитный банк     673
хоум кредит банк              575
Name: bank_name, dtype: int64

In [5]:
df = df[df['bank_name'] == 'сбербанк'] # выбрал Сбербанк

In [6]:
df.shape

(2481, 10)

In [7]:
df.head()

,bank_href,login,review,bank_name,site,date,message_href,score,status,user_href
232,/sberbank-rossii.html,NaN,3 раза я обращался по вопросу списаний за овер...,сбербанк,http://bankireview.ru,NaN,NaN,NaN,NaN,NaN
233,/sberbank-rossii.html,NaN,Очень возмущена и удивлена сложившейся ситуаци...,сбербанк,http://bankireview.ru,NaN,NaN,NaN,NaN,NaN
234,/sberbank-rossii.html,NaN,"Я уже не знаю, дождемся ли мы, чтобы сбер пров...",сбербанк,http://bankireview.ru,NaN,NaN,NaN,NaN,NaN
235,/sberbank-rossii.html,NaN,"Здравствуйте, у меня возникла следующая пробле...",сбербанк,http://bankireview.ru,NaN,NaN,NaN,NaN,NaN
236,/sberbank-rossii.html,NaN,"Дорогой друг Сбербанк, как же мне не хочется и...",сбербанк,http://bankireview.ru,NaN,NaN,NaN,NaN,NaN


#### Лемматизация с помощью pymorphy2

In [8]:
lemmatizer = pymorphy2.MorphAnalyzer()
stop_words = stopwords.words("russian")

In [9]:
all_tokens = [] # здесь будут хранится все токены всех обзоров
def clean_text(text):
    text = re.sub(r"[^\w\s]", "", text, re.UNICODE)
    text = text.lower()
    text = [lemmatizer.parse(token)[0].normal_form for token in text.split(" ") if token.isalpha()]
    for token in text: # здесь явно заменяется часто встречаемый токен "банкхороший"
      if token == 'банкхороший':
        text.remove('банкхороший')
        text.extend(['банк', 'хороший'])
    text = [word for word in text if not word in stop_words]
    all_tokens.extend(text)
    text = " ".join(text)
    return text

In [10]:
df["processed_review"] = df["review"].apply(lambda x: clean_text(x))

In [11]:
df[['processed_review']].iloc[1][0]

'очень возмутить удивить сложиться ситуация сбербанк вчера перевести деньга карта телефон сообщение пополнение баланс прийти баланс факт измениться отделение сбербанк санктпетербург ул типанов смочь помочь посоветовать позвонить круглосуточный служба поддержка разговор оператор выяснить паспортный дать открыть ещё карта деньга поступить именно хотя весь жизнь весь карта сбербанк это мочь сказать уверенность причём карта открыть год вышеупомянутый перевод первый операция неизвестный ранее карта короче говорить оператор кстати переключить свой доброжелательный тон грубоватый расспрос дальнейший рекомендация смочь помочь итог перевести деньга получить заморачиваться заявление непонятно откуда появиться карта тд время это далеко первый неприятный ситуация закрывать всё карта впредь собираться пользоваться услуга данный весь очень разочаровать'

In [12]:
fdist = FreqDist(all_tokens) # подсчет количества всех токенов

In [13]:
fdist.most_common(10) # 10 наиболее частых токенов

[('банк', 4196),
 ('карта', 3938),
 ('сбербанк', 3604),
 ('это', 3506),
 ('деньга', 2190),
 ('всё', 2098),
 ('отделение', 1855),
 ('который', 1828),
 ('сотрудник', 1797),
 ('день', 1506)]

In [14]:
# деление датасета на 2: с позитивными отзывами и негативными отзывами
positive = df.loc[df['score'].isin(('4', '5'))]['processed_review']
negative = df.loc[df['score'].isin(('1', '2'))]['processed_review']

In [15]:
positive.shape

(323,)

In [16]:
negative.shape

(1402,)

In [17]:
# для нахождения прилагательных в контексте слова "банк" используются биграммы
bigrams_positive = []
bigrams_negative = []

bigrams_positive.extend([bigram for review in positive for bigram in list(nltk.bigrams(review.split(' ')))])
bigrams_negative.extend([bigram for review in negative for bigram in list(nltk.bigrams(review.split(' ')))])

In [18]:
positive_adj = []
negative_adj = []

In [19]:
# формируем множество прилагательных в контексте слова "банк" в позитивных отзывах
positive_adj.extend([word for bigram in bigrams_positive for word in bigram
                        if lemmatizer.tag(word)[0].POS in ('ADJF', 'ADJS') and
                        'банк' in bigram])

In [20]:
len(positive_adj)

91

In [21]:
FreqDist(positive_adj).most_common(10) # 10 наиболее частых прилагательных (позитивные)

[('ваш', 8),
 ('мобильный', 8),
 ('большой', 7),
 ('хороший', 7),
 ('весь', 5),
 ('данный', 5),
 ('который', 4),
 ('первый', 3),
 ('удобный', 3),
 ('оперативный', 2)]

In [22]:
# формируем множество прилагательных в контексте слова "банк" в негативных отзывах
negative_adj.extend([word for bigram in bigrams_negative for word in bigram
                        if lemmatizer.tag(word)[0].POS in ('ADJF', 'ADJS') and
                        'банк' in bigram])

In [23]:
len(negative_adj)

747

In [24]:
FreqDist(negative_adj).most_common(10) # 10 наиболее частых прилагательных (негативные)

[('мобильный', 72),
 ('хороший', 59),
 ('который', 46),
 ('ваш', 38),
 ('данный', 31),
 ('свой', 26),
 ('весь', 20),
 ('должный', 19),
 ('наш', 9),
 ('разный', 7)]

pymorphy2 считает местоимения-прилагательные за прилагательные, поэтому далее я попробовал применить pymystem3, так как в нем присутствует разделения между этими граммемами.

---

#### Тот же процесс, но с лемматизатором pymystem3

In [25]:
lemmatizer = Mystem()

In [26]:
df = pd.read_csv('financial_review.csv')
df = df[df['bank_name'] == 'сбербанк']

In [27]:
all_tokens = []
def clean_text(text):
    text = re.sub(r"[^\w\s]", "", text, re.UNICODE)
    text = text.lower()
    text = [lemmatizer.lemmatize(token)[0] for token in text.split(" ") if token.isalpha()]
    for token in text:
      if token == 'банкхороший':
        text.remove('банкхороший')
        text.extend(['банк', 'хороший'])
    text = [word for word in text if not word in stop_words]
    all_tokens.extend(text)
    text = " ".join(text)
    return text

In [28]:
df["processed_review"] = df["review"].apply(lambda x: clean_text(x))

In [29]:
df[['processed_review']].iloc[1][0]

'очень возмущать удивлять складываться ситуация сбербанк вчера переводить деньги карта телефон сообщение пополнение баланс приходить баланс факт изменяться отделение сбербанк санктпетербург ул типановый смочь помогать посоветовать позвонить круглосуточный служба поддержка разговор оператор выяснять паспортный данные открытый карта деньги поступать именно хотя весь жизнь карта сбербанк это мочь сказать уверенность причем карта открытый год вышеупомянутый перевод первый операция неизвестный ранее карта короче говорить оператор кстати переключать свой доброжелательный тон грубоватый расспрос дальнейший рекомендация смочь помогать итог переводить деньги получать заморачиваться заявление непонятно откуда появляться карта тд время это далеко первый неприятный ситуация закрывать карта впредь собираться пользоваться услуга данный очень разочаровывать'

In [30]:
fdist = FreqDist(all_tokens)

In [31]:
fdist.most_common(10) # 10 наиболее частых токенов

[('карта', 3938),
 ('сбербанк', 3605),
 ('это', 3594),
 ('банк', 2399),
 ('деньги', 2183),
 ('отделение', 1855),
 ('который', 1828),
 ('банка', 1797),
 ('сотрудник', 1797),
 ('день', 1506)]

Стоит отметить, что наборы 10-ти наиболее частых слов различаются при использовании разных лемматизаторов.

In [32]:
positive = df.loc[df['score'].isin(('4', '5'))]['processed_review']
negative = df.loc[df['score'].isin(('1', '2'))]['processed_review']

In [33]:
bigrams_positive = []
bigrams_negative = []

bigrams_positive.extend([bigram for review in positive for bigram in list(nltk.bigrams(review.split(' ')))])
bigrams_negative.extend([bigram for review in negative for bigram in list(nltk.bigrams(review.split(' ')))])

In [34]:
positive_adj = []
negative_adj = []

In [35]:
positive_adj.extend([word for bigram in bigrams_positive for word in bigram
                        if lemmatizer.analyze(word)[0]['analysis'] and
                        lemmatizer.analyze(word)[0]['analysis'][0]['gr'][0:2] == 'A=' and
                        'банк' in bigram])

In [36]:
len(positive_adj)

29

In [37]:
FreqDist(positive_adj).most_common(10) # 10 наиболее частых прилагательных (позитивные)

[('хороший', 6),
 ('мобильный', 5),
 ('разный', 2),
 ('сторонний', 2),
 ('полный', 1),
 ('центральный', 1),
 ('российский', 1),
 ('новый', 1),
 ('оперативный', 1),
 ('плохой', 1)]

In [38]:
negative_adj.extend([word for bigram in bigrams_negative for word in bigram
                        if lemmatizer.analyze(word)[0]['analysis'] and
                        lemmatizer.analyze(word)[0]['analysis'][0]['gr'][0:2] == 'A=' and
                        'банк' in bigram])

In [39]:
len(negative_adj)

339

In [40]:
FreqDist(negative_adj).most_common(10) # 10 наиболее частых прилагательных (негативные)

[('хороший', 57),
 ('мобильный', 55),
 ('должный', 8),
 ('разный', 7),
 ('ужасный', 6),
 ('надежный', 6),
 ('обязанный', 5),
 ('полный', 5),
 ('нормальный', 4),
 ('незаконный', 4)]

Одним из наиболее частых прилагательных как в негативных, так и в позитивных отзывах является слово "мобильный", что может говорить о том, что пользователи часто сталкиваются с проблемами при использовании мобильного банка, хотя в то же время он предоставляет некоторые удобные функции. Прилагательное "хороший" в негативных отзывах может относиться к описанию "хороших условий", которые банк не сумел реализовать для пользователя.